In [97]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import ExcelWriter
from pandas import ExcelFile


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['sample']


In [98]:
PATH= '../input/sample/sample.xlsx'

In [99]:
df = pd.read_excel(PATH, sheet_name='Sheet1',reset_index=True)


In [100]:
df = df.drop('sid', 1)

In [101]:
df.head()

,분야,ko,en
0,향토문화/음식,"부산 평화 시장은 1940년 일제 강점기 때, 조선방직 부근에서 정기 시장인 오일장...",The peace market in Busan originated from the ...
1,향토문화/음식,당시 춘양장은 내광비·외광비에서 60리로 거리가 멀어 대부분 가지 않았다고도 한다.,"At that time, most of the people did not go to..."
2,향토문화/음식,그러나 영동고속도로가 개통된 이후 대관령과 경포해수욕장 등이 영화 촬영지로 각광을 ...,"However, after the opening of the Yeongdong Ex..."
3,향토문화/음식,현재 성곽이 있는 산을 넘으면 동해고속도로가 보이고 모전리를 지나 강릉 시내까지 넓...,If you cross the mountain with the current for...
4,향토문화/음식,꽃잎 가운데 2개는 달걀 모양 피침형으로 끝이 뾰족하며 아래쪽에 털이 조금 난다.,"Two of the petals are egg-shaped, lanceolate, ..."


In [6]:
df['분야'].value_counts()

의료/보건      5000
국제스포츠행사    5000
대법원판례      5000
IT/기술      5000
관광         5000
금융/증시      5000
향토문화/음식    5000
Name: 분야, dtype: int64

In [7]:
pd.isnull(df['en']).sum()

0

In [8]:
df.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [9]:
# Lowercase all characters
df['en']=df['en'].apply(lambda x: x.lower())


In [10]:
# Remove quotes
df['en']=df['en'].apply(lambda x: re.sub("'", '', x))
df['ko']=df['ko'].apply(lambda x: re.sub("'", '', x))

In [11]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 를 제거합니다.
df['en']=df['en'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['ko']=df['ko'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))


In [13]:
# Add start and end tokens to target sequences
df['ko'] = df['ko'].apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
df.head()

,분야,ko,en
0,향토문화/음식,START_ 부산 평화 시장은 1940년 일제 강점기 때 조선방직 부근에서 정기 시...,the peace market in busan originated from the ...
1,향토문화/음식,START_ 당시 춘양장은 내광비·외광비에서 60리로 거리가 멀어 대부분 가지 않았...,at that time most of the people did not go to ...
2,향토문화/음식,START_ 그러나 영동고속도로가 개통된 이후 대관령과 경포해수욕장 등이 영화 촬영...,however after the opening of the yeongdong exp...
3,향토문화/음식,START_ 현재 성곽이 있는 산을 넘으면 동해고속도로가 보이고 모전리를 지나 강릉...,if you cross the mountain with the current for...
4,향토문화/음식,START_ 꽃잎 가운데 2개는 달걀 모양 피침형으로 끝이 뾰족하며 아래쪽에 털이 ...,two of the petals are eggshaped lanceolate wit...


In [15]:
### Get eng and kor Vocabulary
all_eng_words=set()
for en in df['en']:
    for word in en.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_kor_words=set()
for ko in df['ko']:
    for word in ko.split():
        if word not in all_kor_words:
            all_kor_words.add(word)

In [16]:
len(all_eng_words)

42562

In [17]:
len(all_kor_words)

168720

In [18]:
df['length_eng_sentence']=df['en'].apply(lambda x:len(x.split(" ")))
df['length_kor_sentence']=df['ko'].apply(lambda x:len(x.split(" ")))

In [19]:
df.head()

,분야,ko,en,length_eng_sentence,length_kor_sentence
0,향토문화/음식,START_ 부산 평화 시장은 1940년 일제 강점기 때 조선방직 부근에서 정기 시...,the peace market in busan originated from the ...,29,19
1,향토문화/음식,START_ 당시 춘양장은 내광비·외광비에서 60리로 거리가 멀어 대부분 가지 않았...,at that time most of the people did not go to ...,27,12
2,향토문화/음식,START_ 그러나 영동고속도로가 개통된 이후 대관령과 경포해수욕장 등이 영화 촬영...,however after the opening of the yeongdong exp...,18,14
3,향토문화/음식,START_ 현재 성곽이 있는 산을 넘으면 동해고속도로가 보이고 모전리를 지나 강릉...,if you cross the mountain with the current for...,25,16
4,향토문화/음식,START_ 꽃잎 가운데 2개는 달걀 모양 피침형으로 끝이 뾰족하며 아래쪽에 털이 ...,two of the petals are eggshaped lanceolate wit...,17,14


In [20]:
df[df['length_eng_sentence']>30].shape

(12980, 5)

In [21]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_kor_sentence']<=20]

In [22]:
df.shape

(8649, 5)

In [23]:
print("maximum length of Korean Sentence ",max(df['length_kor_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Korean Sentence  20
maximum length of English Sentence  20


In [24]:
max_length_src=max(df['length_eng_sentence'])
max_length_tar=max(df['length_kor_sentence'])

In [25]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_kor_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_kor_words)
num_encoder_tokens, num_decoder_tokens

(42562, 168720)

In [26]:
num_decoder_tokens += 1 #for zero padding


In [27]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [28]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [29]:
df = shuffle(df)
df.head(10)

,분야,ko,en,length_eng_sentence,length_kor_sentence
33018,관광,START_ 이번 임시증편은 여름 성수기 좌석난 해소에 도움이 될 것으로 보인다 _END,this temporary increase is expected to help re...,16,12
23695,의료/보건,START_ 발목을 삐끗했다면 전문의를 찾아 진료를 받는 것이 좋습니다 _END,if you have sprained your ankle youd better se...,15,10
857,향토문화/음식,START_ 동 내부에 본성로가 통과하고 중안동과의 경계에 은열로가 있다 _END,bonseongro passes inside the dong and eunyeolr...,13,10
11168,IT/기술,START_ WSJ은 구글의 프로젝트 나이팅게일이 헬스케어 시장에 앞다퉈 진출하려는...,wsj said that googles project nightingale is t...,20,17
25793,국제스포츠행사,START_ 몸에 무리가 갈 수 있어 휴식을 생각하고 있다 _END,im thinking of taking a rest because my body m...,12,10
26782,국제스포츠행사,START_ 권순우는 4세트에서 상대보다 먼저 브레이크에 성공했지만 몸이 버티질 못...,kwon soonwoo succeeded in braking before his o...,19,11
6089,금융/증시,START_ 이에 따라 올해 이마트의 신선식품 분야에선 새롭고 참신한 상품들이 매대...,as a result new and fresh products are expecte...,20,14
34197,관광,START_ 가마골은 담양군 용면 용연리에 소재한 용추봉523m을 중심으로 사방 4...,gamagol valley is a valley formed over 4km in ...,20,15
12459,IT/기술,START_ 와삭바삭 바게뜨는 바삭바삭한 바게트에 고소한 완두콩과 알싸한 고추냉이 ...,crunchy baguette is a combination of nutty pea...,16,13
11219,IT/기술,START_ 인가 회의가 사실상 깜깜이 방식으로 진행됐다는 비판도 나온다 _END,critics say the approval meeting was actually ...,12,10


In [30]:
df.shape

(8649, 5)

### Split the data into train and test

In [31]:
X, y = df['en'], df['ko']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((6919,), (1730,))

In [32]:
y.head()

33018    START_ 이번 임시증편은 여름 성수기 좌석난 해소에 도움이 될 것으로 보인다 _END
23695         START_ 발목을 삐끗했다면 전문의를 찾아 진료를 받는 것이 좋습니다 _END
857          START_ 동 내부에 본성로가 통과하고 중안동과의 경계에 은열로가 있다 _END
11168    START_ WSJ은 구글의 프로젝트 나이팅게일이 헬스케어 시장에 앞다퉈 진출하려는...
25793                START_ 몸에 무리가 갈 수 있어 휴식을 생각하고 있다 _END
Name: ko, dtype: object

### Let us save this data

In [33]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [34]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [35]:
latent_dim=300

In [36]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [37]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [38]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [39]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    12768600    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    50616300    input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [40]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
54/54 [==============================] - 181s 3s/step - loss: 10.2304 - val_loss: 9.8567
Epoch 2/100
54/54 [==============================] - 173s 3s/step - loss: 8.9825 - val_loss: 10.0741
Epoch 3/100
54/54 [==============================] - 172s 3s/step - loss: 8.6669 - val_loss: 10.2250
Epoch 4/100
54/54 [==============================] - 172s 3s/step - loss: 8.4161 - val_loss: 10.3045
Epoch 5/100
54/54 [==============================] - 171s 3s/step - loss: 8.1728 - val_loss: 10.3656
Epoch 6/100
54/54 [==============================] - 173s 3s/step - loss: 7.9537 - val_loss: 10.4615
Epoch 7/100
54/54 [==============================] - 173s 3s/step - loss: 7.7677 - val_loss: 10.5152
Epoch 8/100
54/54 [==============================] - 172s 3s/step - loss: 7.5912 - val_loss: 10.6120
Epoch 9/100
 5/54 [=>............................] - ETA: 2:07 - l

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [8]:
df

,분야,ko,en
0,향토문화/음식,"부산 평화 시장은 1940년 일제 강점기 때, 조선방직 부근에서 정기 시장인 오일장...",The peace market in Busan originated from the ...
1,향토문화/음식,당시 춘양장은 내광비·외광비에서 60리로 거리가 멀어 대부분 가지 않았다고도 한다.,"At that time, most of the people did not go to..."
2,향토문화/음식,그러나 영동고속도로가 개통된 이후 대관령과 경포해수욕장 등이 영화 촬영지로 각광을 ...,"However, after the opening of the Yeongdong Ex..."
3,향토문화/음식,현재 성곽이 있는 산을 넘으면 동해고속도로가 보이고 모전리를 지나 강릉 시내까지 넓...,If you cross the mountain with the current for...
4,향토문화/음식,꽃잎 가운데 2개는 달걀 모양 피침형으로 끝이 뾰족하며 아래쪽에 털이 조금 난다.,"Two of the petals are egg-shaped, lanceolate, ..."
5,향토문화/음식,"작목반 및 부녀회원들이 마을의 대소사를 운영하고 있으며, 마을 발전 및 소득 증대 ...",The members of the crop group and the women's ...
6,향토문화/음식,이에 낙동강이 주요 식수원인 부산 지역 시민들 사이에서 낙동강을 보호하고 환경을 개...,"As a result, movements to protect the Nakdong ..."
7,향토문화/음식,"그 결과 2010년, 경안천의 본류 및 지천 53㎞와 38만 평의 생태 습지가 조성...","As a result, in 2010, an ecological wetland wa..."
8,향토문화/음식,부산여성회 한부모가족자립지원센터[소장 김직상]가 전국의 10개 단체를 연결한 한부모...,The Busan Women Association Single-Parent Fami...
9,향토문화/음식,언론 노동자의 권익 향상과 민주 언론 구현을 위해 전국언론노동조합 부산울산경남협의회...,Recent activities by the Busan-Ulsan South Gye...


In [102]:
df1 = df

In [103]:
df1['keyword'] = df1['en'].str.contains('KRW', regex=True)

In [104]:
df2 = df1[df1['keyword']== True]

In [199]:
df3 = df2

In [200]:
df3 = df3.reset_index()

In [201]:
k=[]
for i in range(len(df3['en'])):
    print(df3.loc[i,'en'].split('KRW')[1].split()[0])

107.2
247.
60,000
31
298.5
2.4
1,237
2.5
20
11,311,000,000.
1.1
2
8,45879,000
48
5.5
1
2.72
460
24.8
10
5
30,000
710
262.5
15,000
4
4.2
322,286,269,000,
2,800,000,
100
132,095
(10,024,000
6.36
15
729.8
448,500.
210
58,680
960
9.627
36.14
262
2
30
870
290
2,500
15
350
4
26.5
700
1.2
20
56
2.877
187.7
216.3
1,525.3
130
in
5.5
129,500
9.827
,
505.5
1.5
10
10
7.86
3.5
1
264,000
996.9
46.7
37.8
3
4
9,209
9.93
3.424
556
19
184.1
1.6
17
46,950,
20
1.9162
1,023,550,
205.07
162.9
37
280
3
288.68
20
4.3516
1
10
269.4
361,
880
143.1
4.7
1.56
210
1
8.9
858.2
8.7766
37.441
523.4
this
600
31,850,
13,150,
which
2
payment,
10
7
194.9
25
1.75
287.7
1
404.8
17.2
220.7
850
6.3
1.5
90.5
900
8.6442
1.3414
10
372.9
260.2
5.3
42.8
40,000
3.8
39,000
14.2
.
2
3.8
227.7
1,890,
3.709
187.4
895.3
47.8
1.97
1,240
2
37.8
223.4
13.7869
2.1
6.9594
4
2
2.1565
10
1
1
3
33,000
11.9
310
59,900),
36,000
1
8.4
32
115,338,900
400
125,000,000
4,800
100
700
46,762,650
70,930,530
75.35
250,000
10
35,588,936.
4,284,574,720
110


In [ ]:
df3['en'] = df3.apply(lambda x: df3['en'].replace('a',df3['value']), axis=1)

In [195]:
for i in range(len(df3)):
    a = df3.loc[0,'en'].split('KRW')[1].split()[0]


107.2
[107.2]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2

[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[1

[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]
[[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]]


In [191]:
df3.loc[0,'en']

"In Jejudo Island, starting in [[107.2]]9[[107.2]][[107.2]] with a square fish reef facility with a width, length, and height each of [[107.2]]m in five village fishing grounds' branch lines with a total size of 83㏊, KRW [[[107.2]][[107.2]][[107.2]][[107.2]][[107.2]]] billion was invested in a total of [[107.2]]4,685㏊ by [[107.2]][[107.2]][[107.2]]5[[107.2]]"

In [40]:
my_string="hello python world , i'm a beginner "
print (my_string.split("world",1)[1] )

 , i'm a beginner 
